In [4]:
# Working on text using machine learning algorithms

sample_train = ['call me tonight', 'Call me cab', 'please call me....PLEASE!']

In [5]:
#vectorising the data words(tokens)
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

vect.fit(sample_train)

# note the parameters inside CountVectorizer() 

/home/ayush/python3_practice/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [6]:
vect.get_feature_names()

['cab', 'call', 'me', 'please', 'tonight']

In [7]:
simple_train_dtm = vect.transform(sample_train)
simple_train_dtm

<3x5 sparse matrix of type '<class 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [8]:
simple_train_dtm.toarray()

array([[0, 1, 1, 0, 1],
       [1, 1, 1, 0, 0],
       [0, 1, 1, 2, 0]])

In [10]:
import warnings
warnings.filterwarnings("always")

import pandas as pd
pd.DataFrame(simple_train_dtm.toarray(), columns=vect.get_feature_names())

,cab,call,me,please,tonight
0,0,1,1,0,1
1,1,1,1,0,0
2,0,1,1,2,0


In [11]:
type(simple_train_dtm)

scipy.sparse.csr.csr_matrix

In [12]:
print(simple_train_dtm)
# insteed of puting all the values it prints the location : value(non zero) pairs 

  (0, 1)	1
  (0, 2)	1
  (0, 4)	1
  (1, 0)	1
  (1, 1)	1
  (1, 2)	1
  (2, 1)	1
  (2, 2)	1
  (2, 3)	2


In [13]:
simple_test = ["please don't call me"]
simple_test_dtm = vect.transform(simple_test)
simple_test_dtm

<1x5 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [14]:
simple_test_dtm.toarray()

array([[0, 1, 1, 1, 0]])

In [15]:
pd.DataFrame(simple_test_dtm.toarray(), columns = vect.get_feature_names())

# note : the word "don't" is removed from the document vector because the model dost not know the relationship 
# between new words(eg. don't) and the label as it is trained according to train data vocabulary

,cab,call,me,please,tonight
0,0,1,1,1,0


In [16]:
# working with real data set
# sms data : contain list of spam or non_spam emails

sms = pd.read_table('data/sms.tsv', header=None, names=['label', 'message'])

In [17]:
sms.shape

(5572, 2)

In [18]:
sms.head(10)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [19]:
sms['label_num'] = sms.label.map({'ham':0, 'spam':1})

In [20]:
sms.head(10)

,label,message,label_num
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
6,ham,Even my brother is not like to speak with me. ...,0
7,ham,As per your request 'Melle Melle (Oru Minnamin...,0
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1


In [21]:
X = sms.message
y = sms.label_num 
print(X.shape)
print(y.shape)

# note : the data variable we created X is 1D because we split it into features using CountVectrisor()
# always make sure that the vector you are going to vectorise is 1D(column vector)

(5572,)
(5572,)


In [22]:
# note : we are going to split data first before vectorising the datasets
# because as during the splitting the data it contains the all real world case(i.e all worlds)
# whereas if we vectorise first it will remove some of the world(s)

from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, stratify=y)

print(X_train.shape)
print(y_train.shape)

(4179,)
(4179,)


/home/ayush/python3_practice/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [23]:
# now vectorising the dataset
vect = CountVectorizer()
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm

# 4179 is the number of samples
# 7373 is the number of words(tokens) present in the vocabulary

<4179x7373 sparse matrix of type '<class 'numpy.int64'>'
	with 55328 stored elements in Compressed Sparse Row format>

In [24]:
X_test_dtm = vect.transform(X_test)
X_test_dtm.shape

(1393, 7373)

In [25]:
# or we can do
X_train_dtm = vect.fit_transform(X_train)
# this is more faster as compare to the privious method

In [26]:
# using Multinmial Naive Bayes classifier 

from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()



In [27]:
%time nb.fit(X_train_dtm, y_train)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 5.16 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [28]:
y_pred = nb.predict(X_test_dtm)

In [29]:
from sklearn import metrics

metrics.accuracy_score(y_test,y_pred)

0.9885139985642498

In [30]:
metrics.confusion_matrix(y_test,y_pred)

#By definition a confusion matrix C is such that C_{i, j} is equal to the number of observations known to be 
#in group i but predicted to be in group j.
#Thus in binary classification, the count of true negatives is C_{0,0}, false negatives is C_{1,0}, 
#true positives is C_{1,1} and false positives is C_{0,1}.

array([[1202,    4],
       [  12,  175]])

In [31]:
# printing out all the false positive

# X_train[(y_pred==1) & (y_test==0)]

# X_train[y_pred > y_test]

In [32]:
# printing all false negative

# X_train[y_pred < y_test]

In [33]:
X[3132] #an example of false negative

"LookAtMe!: Thanks for your purchase of a video clip from LookAtMe!, you've been charged 35p. Think you can do better? Why not send a video in a MMSto 32323."

In [34]:

y_pred_proba = nb.predict_proba(X_test_dtm)[:,1]
y_pred_proba

# note that it is naive bayes probability not actual probability i.e 100% accurate

array([7.82887542e-08, 3.02868734e-08, 1.38606514e-11, ...,
       1.00000000e+00, 1.00000000e+00, 2.62417931e-06])

In [35]:
# calculating AUC score

metrics.roc_auc_score(y_test, y_pred_proba)

0.9876686088275202

In [36]:
# Next Model : Logistic Regression(regularised)

from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

In [39]:
%time logreg.fit(X_train_dtm,y_train)
# compare time with Naive_bayes

CPU times: user 44 ms, sys: 0 ns, total: 44 ms
Wall time: 43 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [40]:
y_pred = logreg.predict(X_test_dtm)

In [44]:
y_pred_proba = logreg.predict_proba(X_test_dtm)[:,1]
y_pred_proba

array([0.02792435, 0.00160332, 0.00434781, ..., 0.99554037, 0.96877537,
       0.0123514 ])

In [45]:
metrics.accuracy_score(y_test,y_pred)

0.9806173725771715

In [46]:
metrics.roc_auc_score(y_test,y_pred_proba)

0.9907148748237422